In [13]:
import ee
import geemap

In [2]:
ee.Initialize(project="nb-lidar")

In [10]:
aoi_id = "projects/nb-lidar/assets/aoi_nb_south"

In [4]:
# extract and export features
from src.features.build_features import build_elevation_features

features = "projects/nb-lidar/assets/nb_south_3_class_bal"

destination = "projects/nb-lidar/assets/exp2/{type}_features"

dsm_features_id = destination.format(type="dsm")
dsm_features = build_elevation_features(
    aoi_id=aoi_id, features_id=features, terrain_type="dsm", dest_id=dsm_features_id
)


dtm_features_id = destination.format(type="dtm")
dtm_features = build_elevation_features(
    aoi_id=aoi_id, features_id=features, terrain_type="dtm", dest_id=dtm_features_id
)


srtm_features_id = destination.format(type="srtm")
srtm_features = build_elevation_features(
    aoi_id=aoi_id, features_id=features, terrain_type="srtm", dest_id=srtm_features_id
)

ee.Reset()

Exporting Features: projects/nb-lidar/assets/exp2/dsm_features
Exporting Features: projects/nb-lidar/assets/exp2/dtm_features
Exporting Features: projects/nb-lidar/assets/exp2/srtm_features


In [8]:
ee.Initialize(project="nb-lidar")

FOLDER_NAME = "nb_lidar_exp2"

# build and assess models
from src.models.train_and_assess import train_smile_random_forest_and_assess
from src.utils.datautils import confusion_matrix_to_drive, monitor_task

dsm_features = ee.FeatureCollection(dsm_features_id)
dsm_rfm, dsm_cfm = train_smile_random_forest_and_assess(features=dsm_features)

dtm_features = ee.FeatureCollection(dtm_features_id)
dtm_rfm, dtm_cfm = train_smile_random_forest_and_assess(features=dtm_features)

srtm_features = ee.FeatureCollection(srtm_features_id)
srtm_rfm, srtm_cfm = train_smile_random_forest_and_assess(features=srtm_features)

# format and export all confusion matrix
filename = "{name}_confusion_matrix"

dsm_cfm_name = filename.format(name="dsm")
confusion_matrix_to_drive(dsm_cfm, folder=FOLDER_NAME, name=dsm_cfm_name)

dtm_cfm_name = filename.format(name="dtm")
confusion_matrix_to_drive(dtm_cfm, folder=FOLDER_NAME, name=dtm_cfm_name)

srtm_cfm_name = filename.format(name="srtm")
confusion_matrix_to_drive(srtm_cfm, folder=FOLDER_NAME, name=srtm_cfm_name)


# save rf models
rf_model_fmt_id = "projects/nb-lidar/assets/exp2/{name}_model"

dsm_model_id = rf_model_fmt_id.format(name="dsm")
task_1 = dsm_rfm.save(asset_id=dsm_model_id)
monitor_task(task_1)

dtm_model_id = rf_model_fmt_id.format(name="dtm")
task_2 = dtm_rfm.save(asset_id=dtm_model_id)
monitor_task(task_1)


srtm_model_id = rf_model_fmt_id.format(name="srtm")
task_3 = srtm_rfm.save(asset_id=srtm_model_id)
monitor_task(task_3)
ee.Reset()

In [18]:
# make predictions
from src.models.predict_model import predict
from src.utils.datautils import classification_to_cloud_stroage

import ee
from ee.data import listAssets

assets = listAssets({"parent": "projects/nb-lidar/assets/exp2"})["assets"]
models = [_.get("id") for _ in assets if _.get("id").endswith("_model")]

dsm, dtm, srtm = models
ee.Initialize(project="nb-lidar")

dsm_precition = predict(aoi_id=aoi_id, model_id=dsm, terrain_type="dsm")
dtm_precition = predict(aoi_id=aoi_id, model_id=dtm, terrain_type="dtm")
srtm_precition = predict(aoi_id=aoi_id, model_id=srtm, terrain_type="srtm")

In [15]:
Map = geemap.Map()
aoi = ee.FeatureCollection(aoi_id).geometry()
vis = {"min": 1, "max": 3}

Map.addLayer(dsm_precition, vis, "DSM Prediction")
Map.addLayer(dtm_precition, vis, "DTM Prediction")
Map.addLayer(srtm_precition, vis, "SRTM Prediction")

Map.centerObject(aoi, 8)

Map

Map(center=[45.74809742262684, -65.99999969409251], controls=(WidgetControl(options=['position', 'transparent_…

In [19]:
BUCKET = "eerfpl-exports"

classification_to_cloud_stroage(
    image=dsm_precition,
    bucket=BUCKET,
    file_name="nb_lidar/exp2/dsm_predci/dsm-",
    region=aoi,
    start=True,
)

classification_to_cloud_stroage(
    image=dtm_precition,
    bucket=BUCKET,
    file_name="nb_lidar/exp2/dtm_predci/dtm-",
    region=aoi,
    start=True,
)

classification_to_cloud_stroage(
    image=srtm_precition,
    bucket=BUCKET,
    file_name="nb_lidar/exp2/srtm_predci/srtm-",
    region=aoi,
    start=True,
)

<Task ZUQR7KELTCPUZBMU66Z7TNGO Type.EXPORT_IMAGE:  (State.UNSUBMITTED)>